# Overview

## Content

[2. Setup](#Setup)

[3. Data Cleaning and Preparation](#Data-Cleaning-&-Preparation)

[4. First Conclusions](#First-Conclusions)

[5. Data Exploration](#Data-Exploration)

[6. Data Cleaning and Preparation](#Data-Cleaning-&-Preparation)

[7. Feature Engineering](#Future-Engineering)

[8. Check Timesseries](#Check-Timeseries)


## Goal: SALES PREDICTION!

`The goal is to predict the revenue generated from different **article categories / Artikelgruppen** in **3 different bakery stores** called Filiale 1, Filiale 2 and Filiale 3.` 

`If the prediction is precise enough, it could prevent food waste and increase profitability for the bakery chain. 
The model is should cover seasonal effects (weekly, yearly) but also incorporates the influence of specific externalities like daily weather conditions or other influential external events like Holidays, Kieler Woche and verkaufsoffene Sonntage.`


## Data



**Filiale** 
- `There are three different filial stores which are analyzed, one direct in the city (filiale=1), one near the water front (filiale=2) and the third in a residential area (filiale=3)`
1 = city; 2 = waterfront area; 3 = city disctrict

**Datum**  `date of the sales data. The period.`
 
**Warengruppe** 
- `main articlegroup`
1 = Brot; 2 = Brötchen; 3 = Spezial_Brötchen; 4 = Konditorei; 5 = Kuchen; 6 = Weihnachtsartikel    

**Artikelgruppe** 
- `article group`
1 = Weizenbrot; 2 = Mischbrot; 3 = Vollkornbrot; 4 = Spezialbrot; 5 = Stangenbrote; 6 = Brötchen; 7 = Süße Brötchen; 8 = Herzhafte Brötchen; 9 = KonditoreiBlech1; 10 = KonditoreiBlech2; 11 = Stückgebäck; 12 = Blechkuchen; 13 = Weihnachtsartikel;     

**Umsatz** `Sales volumne per day and articlegroup`   

# Setup¶

In [2]:
#import basic system functions
import os
import datetime
from collections import Counter

#import libraries for data wrangling
import pandas as pd
import numpy as np

#import libraries for plotting data
import matplotlib.pyplot as plt
import matplotlib.pyplot as mplt

%matplotlib inline
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pandas.plotting import autocorrelation_plot



#import libraries for time series analysis
#from fbprophet import Prophet
from pylab import rcParams
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import grangercausalitytests
from pmdarima.arima import auto_arima

#import libraries for statistics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
#from sklearn.metrics import mean_absolute_percentage_error


import warnings
warnings.filterwarnings('ignore')


from IPython.display import Image
from datetime import datetime

In [2]:
 df= pd.read_csv("/Users/brittarover/Capstone/Meteolytics/meteolytix_Artikelgruppen_Umsatz_verschiedeneStandorte.csv")

# EDA-Summary

You will find [full information about saisonal dependencies](./saisonality.xlsx) in the excel-sheet.

## Filiale 1

**Conclusions FILIALE 1 in total**
- `Sales in total are relatively constant over the year. The revenue on weekdays is a big higher than on weekdays. This is maybe due to the location of the store - the city center of Kiel.`
- `Filiale 1 is closed on sunday with the following exceptions: 1. sunday of every month (not in february), during the Kieler Woche in June, on Verkaufsoffenen Sonntagen and on the last sunday in september. In cas of the 1. sunday beeing a public celebration day, the store is openend on the 2. sunday of the month.`

**Conclusions FILIALE 1 on articlegroups**
- `Sales in total have a correlation to the weekday. This is  valid for most of the articlegroups.`
- `Most of the bread is sold on mondays when the workingweek starts. This is valid for all bread types except Stangenbrote.`
- `The hightest part of Blechkuchen is sold on fridays and saturdays when the workingsweek ends ..`
- `The highest Salesvolumne per Artikelgruppe was 1091,65€ for Artikelgruppe 11:= Stückgebäck on New Year's Eve
2018, everbody buys Berliner for New Year's Eve.`
- `The highest Salesvolumne per Filiale was 4411.73€ with also a hugh salesvolumne for articlegroup 11:= Stückgebäck and a couple of other articlegroups. On this day was a store reopening, the store has been closed for a couple weeks, since the 17th of january.`

- `Trends can be seen for Brötchen (increase), Herzhafte Brötchen (slightly increase), Vollkornbrot (strong increase), Mischbrot (increase) and Stangenbrot and Weizenbrot (slightly decrease). `
- `KonditoreiBlech1 is not sold since the middle of 2019, so I decide to define KonditoreiBlech as sum of both given articlegroups.`


**Impact of Corona**
- `There is an negative impact of corona to the sales values of Filiale 1. The peak goes up to - 40% in week 16 (Lockdown), after that the values are becoming better, but they still remain significant (-20% in week 22) under those of the previous year.`
- `This is valid for all Warenguppen. The development of the corresponding sales values is very similar which makes a difference to the two other stores.`

## Filiale 2

**Conclusions FILIALE 2 in total**
- `Sales in total are relatively constant over the years.`
- `But there is a strong seasonal effect. During summertime they are higher sales for all the years. The peaks depends on the year. Maybe due to the impact of the summer holidays of Schleswig Holstein.`
- `There is also a weekday effect, but not a strong one. The values are a bit higher for saturdays and sundays. The lowest values are on mondays, tuesdays and wednesdays. Kiel is a attractive holiday destination for extended weekendtrips.
`

**Conclusions FILIALE 2 on article groups**
- `Sales in total have a strong correlation to the season and to the weekday. This is  valid for most of the articlegroups.`
- `The biggest impact can be seen for Brötchen, Süße Brötchen, Herzhafte Brötchen and Blechkuchen.`
- `The hightest values are on weekends / sundays and during the summer season. This is valid for all arcticlegroups except bread.`
-  `The highest Salesvolumne per Artikelgruppe was 1798.58€ for Artikelgruppe 11:= Stückgebäck on New Year's Eve
2015, everbody buys Berliner for New Year's Eve. The day with the highest Salesvolumne in total was the 20.04.2019.`
- `Trends can be seen for all Brötchen (increase), Vollkornbrot (strong increase), Mischbrot (increase) and Stangenbrot and Weizenbrot (slightly decrease). `
- `KonditoreiBlech1 is not sold since the beginning of 2019, so I decide to define KonditoreiBlech as sum of both given articlegroups.`

**Impact of Corona**
- `There is an negative impact of corona to the sales values of Filiale 2. The peak goes up to - 35 % in week 16 (Lockdown), after that the values are becoming better. For weak 19,21 and 22 the values 2020 are significant higher than those of the years before. This is due to the end of the Beherberungsverbotes in Schleswig-Holstein. Everybody is looking forward to make a weekend or day trip to the sea side to relax.`
- `This is valid for all Warenguppen. The development of the corresponding sales values is a bit different.`
- `The sales values for bread are the first to recover. But since week 20 there is a run on Herzhafte Brötchen and Brötchen due to the return of the holiday-makers.`


## Filiale 3

**Conclusions FILIALE 3 in total**
- `Sales values in total are relatively constant over the years until Corona starts in march 2020..`
- `But there is a strong relationsship between weekday and over-all salesvalue. For weekends the values are significant higher. This is valid for all the years and most of the articlegroups except all breads.`

**Conclusions FILIALE 3 on article groups**
- `Sales in total have a strong correlation to the weekday. This is  valid for most of the articlegroups.`
- `The biggest impact can be seen for Brötchen, Süße Brötchen, Herzhafte Brötchen and Blechkuchen.`
- `The hightest values are on weekends / sundays. This is valid for all arcticlegroups except bread.`
- `Most of the bread is sold on wednesdays.`
- `The highest Salesvolumne per articlegroup was 3154.15€ for articlegroup 11:= Stückgebäck on New Year's Eve
2017, everbody buys Berliner for New Year's Eve. This is also the day with the highest Salesvolumne in total.`
- `Trends can be seen for all Brötchen (increase), Vollkornbrot (strong increase) and Mischbrot (increase).`
- `KonditoreiBlech1 is not sold since the beginning of 2019, so I decide to define KonditoreiBlech as sum of both given articlegroups.`


**Impact of Corona**
- `There is an negative impact of corona to the sales values of Filiale 3. The peak goes up to - almost -30 % in week 16 (Lockdown), after that the values are becoming better soon. This is valid for all regarded products.`
- `For week 20 the values in 2020 are significant higher than those of the years before due to the hugh sales value of Brötchen.`

- `You can see that Filiale 3 has a peak in weak 20 - different from Filiale 2 (peak in week 19 and 21)`.
- `The sales for Kuchen are the first to recover.`

# Results Predictions

**Overview**
- `Enclosed you will find an overview about the results of the predictions of the analysed timeseries. I focus myself on store 2 / Filiale 2 closed to the waterfront and the belonging timeseries.`
- `I use MAPE and MAE as metric to evalute the results. I adjusted MAPE to cover also the closed values (sales amount:=0).`
- `The models which performed best are: Prophet and SARIMAX and also Random Forest. The exact order depends on the articlegroup.`
- `The results of the VAR / VARMAX are really bad. I think this is due to the fact that there are no strong relationsships between the labels respectively they are not of such a great importance compared to features like sales offers, product placements and so on.`
- `The results of the Expontential Smoothing / Winters Holt are in all cases worse than those of the best models. I think this is due to the fact that WHES can not cover exogene factors like the Sarimax or FB Prophet.`


## Baseline

### Simple Baseline Shift 7

I use shift7 as a very simple first baseline.

In [8]:
df_results_Baseline= pd.read_csv("df_results_Baseline_shift")
df_results_Baseline

,Unnamed: 0,Model,Label,MAE_BASELINE,MAPE_BASELINE
0,0,BASELINE-SHIFT,Weizenbrot,15.933675,21.990651
1,1,BASELINE-SHIFT,Mischbrot,17.654305,23.645570
2,2,BASELINE-SHIFT,Vollkornbrot,22.239644,18.866356
3,3,BASELINE-SHIFT,Stangenbrote,16.366801,49.855707
4,4,BASELINE-SHIFT,Spezialbrot,18.415565,26.599027
5,5,BASELINE-SHIFT,Brötchen,69.431490,16.911998
6,6,BASELINE-SHIFT,Süsse_Brötchen,29.227411,13.062030
7,7,BASELINE-SHIFT,Herzhafte_Brötchen,23.752415,24.828228
8,8,BASELINE-SHIFT,KonditoreiBlech,35.967063,42.107328
9,9,BASELINE-SHIFT,Stückgebäck,46.763597,27.807253


### Advanced Baseline - Moving Average

A moving average value with window:=7 can also be used directly to make predictions as a naive model and baseline for further 
on analysis. 

In [147]:
df_results_Baseline_adv= pd.read_csv("df_results_Baseline_adv")
df_results_Baseline_adv.drop(columns=['Unnamed: 0'],inplace=True)
df_results_Baseline_adv

,Model,Label,MAE_BASELINE,MAPE_BASELINE
0,BASELINE,Weizenbrot,21.810966,18.654254
1,BASELINE,Mischbrot,14.696344,19.716579
2,BASELINE,Vollkornbrot,17.744443,14.401461
3,BASELINE,Stangenbrote,13.188681,40.556283
4,BASELINE,Spezialbrot,16.508831,20.319135
5,BASELINE,Brötchen,77.563061,18.530199
6,BASELINE,Süsse_Brötchen,32.109803,13.787416
7,BASELINE,Herzhafte_Brötchen,18.289108,18.341505
8,BASELINE,KonditoreiBlech,34.783809,39.552476
9,BASELINE,Stückgebäck,36.841229,20.410318


## Best Results Sarimax

In [14]:
df_results_SARIMAX= pd.read_csv("df_results_Sarimax.csv")
df_results_SARIMAX.drop(columns=['Unnamed: 0'],inplace=True)

df_results_SARIMAX

,Model,Label,MAE_NEW,MAPE_NEW
0,SARIMAX_BOXCOX,Weizenbrot,20.291094,15.474022
1,SARIMAX_BOXCOX,Mischbrot,16.232547,19.685878
2,SARIMAX_BOXCOX,Vollkornbrot,18.097057,12.429291
3,SARIMAX_BOXCOX,Spezialbrot,11.356501,15.948933
4,SARIMAX-CF,Stangenbrote,12.930264,36.876216
5,SARIMAX_BOXCOX,Brötchen,61.373127,12.385237
6,SARIMAX-CF,SüsseBrötchen,25.908363,11.215817
7,SARIMX_BOXCOX,Herzhafte_Brötchen,21.592032,16.793494
8,SARIMAX_BOXCOX,KonditoreiBlech,17.227928,22.534191
9,SARIMX-CF,Stückgebäck,37.258637,22.881809


## Best Results FB Prophet

**Overview**
- `Most of the predictions are quite good in case of running FB Prophet without any adjustments. This is valid for mostly bread types and for Brötchen and Süsse Brötchen.`

### Prophet Base without adjustments

In [33]:
df_results_Prophet= pd.read_csv("df_results_Prophet_Basis.csv")
df_results_Prophet.drop(columns=['Unnamed: 0'],inplace=True)

df_results_Prophet

,Model,Label,MAE,MAPE
0,Prophet-Basis,Weizenbrot,13.180610,18.006223
1,Prophet-Basis,Mischbrot,13.821258,17.954490
2,Prophet-Basis,Vollkornbrot,18.469717,13.721410
3,Prophet-Basis,Stangenbrote,11.840483,34.313274
4,Prophet-Basis,Spezialbrot,12.685970,18.715479
5,Prophet-Basis,Brötchen,54.584405,12.403264
6,Prophet-Basis,Süsse_Brötchen,23.920164,9.929626
7,Prophet-Basis,Herzhafte_Brötchen,25.618234,24.742623
8,Prophet-Basis,KonditoreiBlech,20.236546,28.700255
9,Prophet-Basis,Stückgebäck,67.695977,47.912907


In [57]:
df_results_Prophet_saison= pd.read_csv("Dateien/df_results_Prophet_Season",delimiter=";")
df_results_Prophet_saison.drop(columns=['Unnamed: 0'],inplace=True)

df_results_Prophet_saison

,Model,Label,MAPE
0,Prophet_Season_DWYmult,Weizenbrot,17.562396
1,Prophet_Season_WYadd,Mischbrot,17.983037
2,Prophet_Season_Diverse,Vollkornbrot,13.724895
3,Prophet_Season_WYmult,Spezialbrot,18.609000
4,Prophet_Season_WYmult,Stangenbrote,33.810000
5,Prophet_Season_WYMult,Brötchen,10.937751
6,Prophet_Season_DWMult,Süsse_Brötchen,9.817811
7,Prophet_Season_WYMult,Herzhafte_Brötchen,23.039517
8,Prophet_Season_Ddiv,KonditoreiBlech,24.161428
9,Prophet_Season_DWYmult,Stückgebäck,26.298908


In [70]:
df_results_Prophet_OPT= pd.read_csv("df_results_Prophet_OPT.csv")
df_results_Prophet_OPT.drop(columns=['Unnamed: 0'],inplace=True)

df_results_Prophet_OPT

,Model,Label,MAE,MAPE
0,Prophet-OPT-W,Vollkornbrot,16.063742,12.403363
1,Prophet-OPT-W,Brötchen,44.898912,10.378933
2,Prophet-OPT,Süsse_Brötchen,19.969256,8.525503
3,Prophet-OPT,Herzhafte_Brötchen,20.009548,17.083921
4,Prophet-OPT,Stückgebäck,35.237249,23.431609
5,Prophet-OPT,Weihnachtsartikel,5.834186,16.308981


## Best Results Random Forest

In [23]:
df_results_RF= pd.read_csv("df_results_Random_Forest_basis.csv")
df_results_RF.drop(columns=['Unnamed: 0'],inplace=True)

df_results_RF

,Model,Label,MAE,MAPE
0,Random_Forest-Basis,Weizenbrot,11.261959,19.946602
1,Random_Forest-Basis,Mischbrot,13.947608,20.771600
2,Random_Forest-Basis,Vollkornbrot,24.599089,19.608866
3,Random_Forest-Basis,Spezialbrot,22.838269,36.398070
4,Random_Forest-Basis,Stangenbrote,14.708428,48.691471
5,Random_Forest-Basis,Brötchen,59.528474,14.329336
6,Random_Forest-Basis,Süsse Brötchen,23.653759,11.381310
7,Random_Forest-Basis,Herzhafte_Brötchen,30.357631,26.763359
8,Random_Forest-Basis,KonditoreiBlech,42.822323,67.054933
9,Random_Forest-Basis,Stückgebäck,41.851936,28.189264


In [29]:
df_results_RF= pd.read_csv("Dateien/df_results_Random_Forest.csv")
df_results_RF.drop(columns=['Unnamed: 0'],inplace=True)

df_results_RF.sort_values(['Label','MAPE'])

,Model,Label,MAE,MAPE
4,Random_Forest-Advanced_inclGesamt,Brötchen,56.783599,13.682035
6,Random_Forest-Advanced_Closedrepl,Brötchen,67.503417,14.099044
2,Random_Forest-Advanced_year,Brötchen,59.904328,14.757452
1,Random_Forest-Advanced,Herzhafte_Brötchen,30.849658,26.977483
8,Random_Forest-Advanced,Herzhafte_Brötchen,30.849658,26.977483
0,Random_Forest-Advanced,Süsse Brötchen,22.170843,10.802429
5,Random_Forest-Advanced_Gesamt,Süsse Brötchen,22.403189,10.936203
7,Random_Forest-Advanced_Closedrepl,Süsse Brötchen,27.011390,11.132135
3,Random_Forest-Advanced_year,Süsse Brötchen,23.248292,11.243179


## Best Results Winters Holt

**Overview**
- `The results for the breadtypes are comparable to those of the other models, but the values for Brötchen and Süsse Brötchen which are the most important articlegroups are even worse.`
- `This is valid for both timeperiods.`

In [20]:
df_results_WH=pd.read_csv("df_results_Winters_Holt_OPT.csv")
df_results_WH.drop(columns=['Unnamed: 0'],inplace=True)

df_results_WH.query('Model=="Winters_Holt"')

,Model,Fit,Label,MAE,MAPE
0,Winters_Holt,Seas_mult,Weizenbrot,20.194921,18.062900
1,Winters_Holt,Seas_mult/Trend_dampF,Mischbrot,17.199704,23.129504
2,Winters_Holt,Seas_add,Vollkornbrot,21.973916,16.727852
3,Winters_Holt,Seas_add/Trend_damp,Spezialbrot,11.203307,15.361160
4,Winters_Holt,Seas_mult/Trend_damp,Stangenbrote,13.725387,38.977111
5,Winters_Holt,Seas_add,Brötchen,87.610231,26.062649
6,Winters_Holt,Seas_mult/Trend_damp,Süsse_Brötchen,38.229343,19.018603
7,Winters_Holt,Seas_mult/Trend_damp,Herzhafte_Brötchen,19.695785,24.797428
8,Winters_Holt,Seas_add,KonditoreiBlech,28.026944,31.725754
9,Winters_Holt,Seas_mult/Trend_damp,Stückgebäck,44.120445,29.727555


In [21]:
df_results_WH.query('Model=="Winters_Holt_long"')

,Model,Fit,Label,MAE,MAPE
11,Winters_Holt_long,Seas_mult/Trend_damp,Weizenbrot,20.194921,18.062900
12,Winters_Holt_long,Seas_mult/Trend_dampF,Mischbrot,17.466539,22.675452
13,Winters_Holt_long,Seas_add,Vollkornbrot,22.869689,16.910162
14,Winters_Holt_long,Seas_add/Trend_damp,Spezialbrot,12.722809,16.877337
15,Winters_Holt_long,Seas_add,Stangenbrote,15.404409,45.599768
16,Winters_Holt_long,Seas_add/Trend_damp,Brötchen,98.655831,22.341864
17,Winters_Holt_long,Seas_mult/Trend_damp,Süsse_Brötchen,39.964015,17.591097
18,Winters_Holt_long,Seas_mult/Trend_damp,Herzhafte_Brötchen,29.237493,25.582229
19,Winters_Holt_long,Seas_add,KonditoreiBlech,23.591092,31.462194
20,Winters_Holt_long,Seas_add,Blechkuchen,39.148405,46.973461


## Best Results VAR/VARMAX

In [67]:
df_results_VAR=pd.read_csv("Dateien/df_results_VAR")
df_results_VAR.drop(columns=['Unnamed: 0','Unnamed: 6'],inplace=True)

df_results_VAR[['Model','Label','MAE','MAPE']]

,Model,Label,MAE,MAPE
0,VARMA-28-long,Weizenbrot,25.392433,81.803024
1,VARMA-28-long,Mischbrot,17.390.228,24.059.806
2,VARMA-28-long,Vollkornbrot,22.163.001,19.416.847
3,VARMA-28-long,Spezialbrot,11.428.301,17.996.767
4,VARMA-28-long,Stangenbrote,13.311.750,34.069.262
5,VARMA-28-long,Brötchen,115.798.833,30.716.290
6,VARMA-28-long,Süsse_Brötchen,45.056.058,22.667.177
7,VARMA-28-long,Herzhafte_Brötchen,30.267.954,29.293.641
8,VARMA-28-long,KonditoreiBlech,23.571.854,28.860.966
9,VARMA-28-long,Stückgebäck,40.904.204,26.419.514


## Best Precition for Brötchen

In [206]:
df_results_Brötchen_OPT=pd.read_csv("Dateien/df_results_Brötchen_OPT.csv")
df_results_Brötchen_OPT.drop(columns=['Unnamed: 0'],inplace=True)

In [207]:
df_results_Brötchen_OPT

,Model,Label,MAE,MAPE,Baseline
0,Baseline_Shift_7,Brötchen,69.431900,16.911990,1
1,Baseline_Moving_Average,Brötchen,77.563010,18.530190,1
2,SARIMAX,Brötchen,61.373127,12.385237,0
3,PROPHET,Brötchen,44.489891,10.378933,0
4,VAR,Brötchen,115.798833,30.716290,0
5,Random Forest,Brötchen,56.072900,13.535600,0
6,Winters Holt,Brötchen,104.805000,29.195400,0


In [203]:
df_results_Brötchen_OPT.to_csv("Dateien/df_results_Brötchen_OPT.csv")

In [127]:
colors = ['seashell',] * 54
colors[1] = 'crimson'

fig = go.Figure(data=[go.Bar(x=df_results_Brötchen_OPT.query('Baseline==0').Model,
                             y=df_results_Brötchen_OPT.query('Baseline==0').MAPE, marker_color=colors,
                             text=df_results_Brötchen_OPT.query('Baseline==0').MAPE
                            
             #color_discrete_sequence=px.colors.diverging.balance
            )])
fig.add_shape( # add a horizontal "target" line
    type="line", line_color="salmon", line_width=3, opacity=1, line_dash="dot",
    x0=0, x1=1, xref="paper", y0=16.912, y1=16.912, yref="y"
)
fig.add_annotation( # add a text callout with arrow
    text="Best Model", x="PROPHET", y=12, arrowhead=1, showarrow=True
)
fig.update_layout(title_text='Modelcomparision for Brötchen-MAPE', title_x=0.01)
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.95,
    xanchor="left",
    x=0.75
))
fig.update_layout(title=dict(
    yanchor="top",
    y=0.88,
    xanchor="left",
    x=0.13))
fig.update_layout( # customize font and legend orientation & position
    font_family="Rockwell",
    legend=dict(
        title=None, orientation="h", y=1, yanchor="bottom", x=0.3, xanchor="center"
    )
)
fig.update_traces(texttemplate='%{text:.4s}', textposition='outside')
fig.update_layout(
    autosize=False,
    width=600,
    height=600,)
fig.show()

## Best Precition for Süsse Brötchen

In [209]:
df_results_Süsse_Brötchen_OPT=pd.read_csv("Dateien/df_results_Süsse_Brötchen_OPT.csv")
df_results_Süsse_Brötchen_OPT.drop(columns=['Unnamed: 0'],inplace=True)

In [210]:
df_results_Süsse_Brötchen_OPT

,Model,Label,MAE,MAPE,Baseline
0,Baseline_Shift_7,Süsse_Brötchen,29.227411,13.062030,1
1,Baseline_Moving_Average,Süsse_Brötchen,32.109803,13.787416,1
2,SARIMAX,Süsse_Brötchen,25.908363,11.215817,0
3,PROPHET,Süsse_Brötchen,19.969256,8.525503,0
4,VAR,Süsse_Brötchen,45.056058,22.667177,0
5,Random Forest,Süsse_Brötchen,23.653759,11.381310,0
6,Winters Holt,Süsse_Brötchen,38.229343,19.018603,0


In [208]:
df_results_Süsse_Brötchen_OPT.to_csv("Dateien/df_results_Süsse_Brötchen_OPT.csv")

In [142]:
colors = ['seashell',] * 54
colors[1] = 'crimson'

fig = go.Figure(data=[go.Bar(x=df_results_Süsse_Brötchen_OPT.query('Baseline==0').Model,
                             y=df_results_Süsse_Brötchen_OPT.query('Baseline==0').MAPE, marker_color=colors,
                             text=df_results_Süsse_Brötchen_OPT.query('Baseline==0').MAPE
                            
             #color_discrete_sequence=px.colors.diverging.balance
            )])
fig.add_shape( # add a horizontal "target" line
    type="line", line_color="salmon", line_width=3, opacity=1, line_dash="dot",
    x0=0, x1=1, xref="paper", y0=13.062, y1=13.062, yref="y"
)
fig.add_annotation( # add a text callout with arrow
    text="Best Model", x="PROPHET", y=10, arrowhead=1, showarrow=True
)
fig.update_layout(title_text='Modelcomparision for Süsse Brötchen-MAPE', title_x=0.01)
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.95,
    xanchor="left",
    x=0.75
))
fig.update_layout(title=dict(
    yanchor="top",
    y=0.88,
    xanchor="left",
    x=0.13))
fig.update_layout( # customize font and legend orientation & position
    font_family="Rockwell",
    legend=dict(
        title=None, orientation="h", y=1, yanchor="bottom", x=0.3, xanchor="center"
    )
)
fig.update_traces(texttemplate='%{text:.4s}', textposition='outside')
fig.update_layout(
    autosize=False,
    width=600,
    height=600,)
fig.show()

## Best Precdictions for Bread

In [ ]:
df_results_Bread=pd.read_csv("df_results_Bread.csv")
df_results_Bread.drop(columns=['Unnamed: 0'],inplace=True)

In [164]:
df_results_Bread['Group']=df_results_Bread['Label'].apply(lambda x: 1 if x in 
                                                     ('Bread','Bread_Baguette') else 0)
df_results_Bread['Baguette']=df_results_Bread['Label'].apply(lambda x: 1 if x in 
                                                     ('Stangenbrote','Bread_Baguette') else 0)

In [165]:
df_results_Bread

,Model,Label,MAPE,Baseline,Group,Baguette
0,SARIMAX,Weizenbrot,15.474022,18.654254,0,0
1,PROPHET,Mischbrot,17.954490,19.716579,0,0
2,PROPHET,Vollkornbrot,12.403363,14.401462,0,0
3,SARIMAX,Spezialbrot,15.948933,20.319135,0,0
4,PROPHET,Stangenbrote,33.810000,40.556283,0,1
5,SARIMAX,Bread,8.235819,15.197618,1,0
6,SARIMAX,Bread_Baguette,13.511671,14.904372,1,1


In [188]:
colors = ['seashell',] * 5
colors[4] = 'crimson'

fig = go.Figure(data=[go.Bar(x=df_results_Bread.query('Baguette==0').Label,
                             y=df_results_Bread.query('Baguette==0').MAPE, marker_color=colors,
                             text=df_results_Bread.query('Baguette==0').MAPE, name='MAPE in %'
                            
             #color_discrete_sequence=px.colors.diverging.balance
            )])
fig.add_trace(go.Bar(x=df_results_Bread.query('Baguette==0').Label,
                             y=df_results_Bread.query('Baguette==0').Baseline,
                             text=df_results_Bread.query('Baguette==0').Baseline, name='Baseline'
                            
             #color_discrete_sequence=px.colors.diverging.balance
            ))
fig.add_shape( # add a horizontal "target" line
    type="line", line_color="salmon", line_width=3, opacity=1, line_dash="dot",
    x0=0, x1=1, xref="paper", y0=8.236, y1=8.236, yref="y"
)
fig.add_annotation( # add a text callout with arrow
    text="Best Model", x="Bread", y=10
)
fig.update_layout(title_text='Best Precitions for all Breads-MAPE versus Baseline', title_x=0.01)
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.95,
    xanchor="left",
    x=0.1
))
fig.update_layout(title=dict(
    yanchor="top",
    y=0.92,
    xanchor="left",
    x=0.13))
fig.update_layout( # customize font and legend orientation & position
    font_family="Rockwell",
    legend=dict(
        title=None, orientation="h", y=1, yanchor="bottom", x=0.3, xanchor="center"
    )
)
fig.update_traces(texttemplate='%{text:.4s}', textposition='outside')
fig.update_layout(
    autosize=False,
    width=600,
    height=600,)
fig.show()

## Best Precition for Cakes

In [201]:
df_results_Cakes_OPT=pd.read_csv("Dateien/df_results_Cakes_OPT.csv")
df_results_Cakes_OPT.drop(columns=['Unnamed: 0'],inplace=True)

In [202]:
df_results_Cakes_OPT

,Model,Label,Baseline,MAPE,Group,Cake
0,SARIMAX,KonditoreiBlech,22.534191,16.911990,0,1
1,SARIMAX,Stückgebäck,20.410318,22.881809,0,1
2,PROPHET,Blechkuchen,30.168072,26.359640,0,1
3,PROPHET,Weihnachtsartikel,20.465365,16.308981,0,0
4,SARIMAX,Herzhafte_Brötchen,18.341505,16.793494,0,0
5,SARIMAX,Cakes,17.622065,16.981459,1,1


In [213]:
colors = ['seashell',] * 5
colors[4] = 'crimson'

fig = go.Figure(data=[go.Bar(x=df_results_Cakes_OPT.query('Cake==1').Label,
                             y=df_results_Cakes_OPT.query('Cake==1').MAPE, marker_color=colors,
                             text=df_results_Cakes_OPT.query('Cake==1').MAPE, name='MAPE in %'
                            
             #color_discrete_sequence=px.colors.diverging.balance
            )])
fig.add_trace(go.Bar(x=df_results_Cakes_OPT.query('Cake==1').Label,
                             y=df_results_Cakes_OPT.query('Cake==1').Baseline,
                             text=df_results_Cakes_OPT.query('Cake==1').Baseline, name='Baseline'
                            
             #color_discrete_sequence=px.colors.diverging.balance
            ))

fig.update_layout(title_text='Best Precitions for all Cakes-MAPE versus Baseline', title_x=0.01)
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.95,
    xanchor="left",
    x=0.1
))
fig.update_layout(title=dict(
    yanchor="top",
    y=0.92,
    xanchor="left",
    x=0.13))
fig.update_layout( # customize font and legend orientation & position
    font_family="Rockwell",
    legend=dict(
        title=None, orientation="h", y=1, yanchor="bottom", x=0.3, xanchor="center"
    )
)
fig.update_traces(texttemplate='%{text:.4s}', textposition='outside')
fig.update_layout(
    autosize=False,
    width=600,
    height=600,)
fig.show()